<a href="https://colab.research.google.com/github/arumlim/eileen/blob/master/03_%E1%84%86%E1%85%A9%E1%84%86%E1%85%A6%E1%86%AB%E1%84%90%E1%85%A5%E1%86%B7_%E1%84%90%E1%85%AE%E1%84%8C%E1%85%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모멘텀 투자

#### 상대모멘텀

In [ ]:
!pip install finance-datareader

In [ ]:
import pandas as pd 

# 구글 드라이브 경로에서 read하기 위한 경로로 수정합니다.
def chage_google_drive(drive_path):
  result_path = url2='https://drive.google.com/uc?id=' + drive_path.split('/')[-2]
  return result_path

# data의 공유 path를 불러옵니다.
share_path_2020 = 'https://drive.google.com/file/d/1x5C194FKhGSvzM8F9qj_J4cEJBSDe8lZ/view?usp=sharing'

# 데이터를 read합니다.
KRX = pd.read_csv(chage_google_drive(share_path_2020), index_col = [0], dtype={'Code':object})

In [ ]:
start_date = KRX.index.min() # 수익률을 계산하기 위한 시작날짜
end_date = KRX.index.max() # 수익률을 계산하기 위한 종료날짜
print(f'시작날짜 : {start_date} \n종료날짜 : {end_date}')

시작날짜 : 2020-01-02 
종료날짜 : 2020-12-30


In [ ]:
data2020 = KRX.reset_index()
data2020.head()

,Date,Open,High,Low,Close,Volume,Change,Name,Code,Sector
0,2020-01-02,2820,2845,2780,2795,131742,-0.008865,3S,060310,전자부품 제조업
1,2020-01-03,2795,2825,2775,2810,100374,0.005367,3S,060310,전자부품 제조업
2,2020-01-06,2800,2805,2650,2685,201779,-0.044484,3S,060310,전자부품 제조업
3,2020-01-07,2685,2745,2685,2745,62241,0.022346,3S,060310,전자부품 제조업
4,2020-01-08,2720,2745,2580,2585,203909,-0.058288,3S,060310,전자부품 제조업


In [ ]:
KRX_pivot = data2020.pivot(index= 'Date', columns = 'Code', values = 'Close')
KRX_pivot = KRX_pivot.loc[start_date:end_date,:]
KRX_PROFIT = KRX_pivot.pct_change() # 수익률 계산
KRX_PROFIT_ACC = (1+KRX_PROFIT).cumprod()-1 # 누적 수익률 계산
KRX_PROFIT_ACC.head()

Code,000020,000040,000050,000060,000070,000080,000100,000120,000140,000150,000180,000210,000220,000230,000240,000250,000270,000300,000320,000370,000390,000400,000430,000440,000480,000490,000500,000520,000540,000590,000640,000650,000660,000670,000680,000700,000720,000760,000810,000850,...,347000,347740,347770,347860,347890,348150,348210,348350,348950,350520,351320,352770,352820,352940,353200,353810,354200,356860,357120,357250,357550,357780,363280,365550,900070,900100,900110,900140,900250,900280,900310,900340,950110,950130,950140,950160,950170,950180,950190,950200
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.004762,-0.010480,0.001044,0.000000,-0.008915,-0.012111,-0.004311,-0.029508,0.007576,-0.008708,0.006173,0.000000,-0.004375,-0.013598,-0.007143,-0.031637,-0.011765,-0.003881,0.000,0.000000,-0.003960,-0.002421,0.001466,0.204762,0.000000,-0.008726,-0.011390,0.004751,0.006494,0.001706,0.004831,-0.001433,-0.002112,0.018576,0.012987,-0.003035,-0.004840,-0.012903,-0.006289,0.001238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001429,0.000000,0.0,-0.001425,-0.002778,0.056075,-0.002577,-0.018182,-0.017734,0.004717,0.050336,0.0,-0.001121,0.048048,NaN,NaN
2020-01-06,-0.026190,-0.032314,-0.004175,-0.029155,-0.046062,-0.008651,-0.021600,-0.036066,0.022727,-0.033382,-0.021605,-0.042137,-0.052330,-0.040604,-0.007143,-0.063274,-0.014118,-0.019405,-0.024,-0.032200,-0.016832,-0.012107,-0.014663,0.400000,-0.032584,-0.034904,-0.043280,-0.042755,-0.003247,-0.003413,-0.014493,-0.011461,-0.004224,0.009288,0.010823,-0.006070,-0.041261,-0.022581,-0.018868,-0.022277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002857,-0.038278,0.0,-0.026511,-0.033333,0.031153,-0.020619,-0.040909,-0.039409,-0.021226,0.051454,0.0,-0.041480,0.018018,NaN,NaN
2020-01-07,-0.028571,-0.060262,-0.002088,-0.043732,-0.035661,-0.015571,-0.004311,-0.036066,0.022727,-0.037736,-0.012346,-0.029608,-0.026249,-0.022568,-0.003571,-0.068776,-0.010588,-0.012937,-0.024,-0.030411,-0.011881,-0.026634,-0.014663,0.195238,-0.042697,-0.033159,-0.075171,-0.028504,-0.006494,-0.013652,-0.028986,-0.012894,-0.007392,0.018576,0.012987,-0.009105,-0.036397,-0.025806,-0.006289,-0.025990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007143,-0.028708,0.0,0.005986,0.000000,0.037383,0.038660,-0.020455,-0.014778,-0.018868,0.058166,0.0,-0.048206,0.021021,NaN,NaN
2020-01-08,-0.055952,-0.074236,-0.032359,-0.058309,-0.083210,-0.041522,-0.023744,-0.059016,0.018939,-0.059507,-0.043210,-0.059216,-0.065455,-0.063078,-0.021429,-0.108666,-0.028235,-0.036223,-0.064,-0.080501,-0.031683,-0.058111,-0.032258,0.185714,-0.070787,-0.057592,-0.109339,-0.078385,-0.024351,-0.032423,-0.046377,-0.025788,0.028511,-0.003096,0.008658,-0.031866,-0.086165,-0.051613,-0.025157,-0.050743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005714,-0.071770,0.0,-0.066420,-0.061111,-0.021807,-0.010309,-0.081818,-0.041379,-0.075472,0.036913,0.0,-0.094170,-0.009009,NaN,NaN


In [ ]:
# 수익률 계산
result_PROFIT_ACC = KRX_PROFIT_ACC.loc[end_date,:].dropna()
result_PROFIT_ACC = result_PROFIT_ACC.sort_values(ascending = False).to_frame().rename(columns = {end_date:'수익률'})
result_PROFIT_ACC = result_PROFIT_ACC.reset_index()
result_PROFIT_ACC

,Code,수익률
0,019170,15.939891
1,323990,14.708920
2,109820,11.183099
3,950130,9.400943
4,058110,8.682799
...,...,...
2106,009730,-0.832407
2107,058220,-0.837758
2108,045890,-0.846975
2109,109070,-0.863606


In [ ]:
result_PROFIT_ACC.head()

,Code,수익률
0,019170,15.939891
1,323990,14.708920
2,109820,11.183099
3,950130,9.400943
4,058110,8.682799


In [ ]:
merge_PROFIT_ACC = result_PROFIT_ACC.merge(KRX[['Name','Code']], on = 'Code', how = 'left').drop_duplicates()
merge_PROFIT_ACC.head()

,Code,수익률,Name
0,019170,15.939891,신풍제약
248,323990,14.708920,박셀바이오
315,109820,11.183099,진매트릭스
563,950130,9.400943,엑세스바이오
811,058110,8.682799,멕아이씨에스


In [ ]:
TOP100_stock = merge_PROFIT_ACC.head(100)['Code'].to_list()


### 2020년 종목에 대해서 21년 11월 까지 보유한다고 할때 수익률을 계산해보자

In [ ]:
# data의 공유 path를 불러옵니다.
share_path_2021 = 'https://drive.google.com/file/d/1ATgr3wbBC4ZCZH-22Ic-U56sBPzz69nB/view?usp=sharing'

# 데이터를 read합니다.
KRX_2021 = pd.read_csv(chage_google_drive(share_path_2021), index_col = [0], dtype={'Code':object}).reset_index()

In [ ]:
print(KRX_2021['Date'].min())
print(KRX_2021['Date'].max())

2021-01-04
2021-12-10


In [ ]:
TOP100_2021 = KRX_2021[(KRX_2021['Code'].isin(TOP100_stock)) ]

In [ ]:
TOP100_2021_pivot = TOP100_2021.pivot(index = 'Date', columns = 'Code', values = 'Close')
buy_price = TOP100_2021_pivot.iloc[0]
sell_price = TOP100_2021_pivot.iloc[-1]
return_price = sell_price - buy_price
TOP100_PROFIT_ACC = ((1+TOP100_2021_pivot.pct_change()).cumprod()-1).iloc[-1]

# last_date = TOP100_PROFIT_ACC.index[-1]
# TOP100_PROFIT_ACC = TOP100_PROFIT_ACC.iloc[-1].sort_values(ascending = False).to_frame().rename(columns = {last_date:'수익률'})
# TOP100_PROFIT_ACC = TOP100_PROFIT_ACC.reset_index()

In [ ]:
TOP100_PROFIT_ACC.sort_values(ascending = False)

Code
058970    2.219409
066970    1.798467
247540    1.774038
232680    1.539855
013720    1.349842
            ...   
109820   -0.601923
323990   -0.684453
019170   -0.722951
012600   -0.738720
156100   -0.743649
Name: 2021-12-10, Length: 100, dtype: float64

In [ ]:
TOP100 = pd.DataFrame({'buy':buy_price, 'sell':sell_price,
              'return':return_price,'rate_of_return' : TOP100_PROFIT_ACC})
TOP100

,buy,sell,return,rate_of_return
Code,,,,
001470,4525,2255,-2270,-0.501657
003090,53200,31900,-21300,-0.400376
005320,4085,3240,-845,-0.206854
006280,475000,237000,-238000,-0.501053
007860,9990,14100,4110,0.411411
...,...,...,...,...
297090,34850,18050,-16800,-0.482066
323990,217400,68600,-148800,-0.684453
336260,56300,47700,-8600,-0.152753


In [ ]:
merge_TOP100 = TOP100.merge(KRX_2021[['Code','Name']],on = 'Code', how = 'left').drop_duplicates()
merge_TOP100

,Code,buy,sell,return,rate_of_return,Name
0,001470,4525,2255,-2270,-0.501657,삼부토건
234,003090,53200,31900,-21300,-0.400376,대웅
468,005320,4085,3240,-845,-0.206854,국동
702,006280,475000,237000,-238000,-0.501053,녹십자
936,007860,9990,14100,4110,0.411411,서연
...,...,...,...,...,...,...
22230,297090,34850,18050,-16800,-0.482066,씨에스베어링
22464,323990,217400,68600,-148800,-0.684453,박셀바이오
22698,336260,56300,47700,-8600,-0.152753,두산퓨얼셀
22932,353810,6200,4615,-1585,-0.255645,이지바이오


In [ ]:
merge_TOP100['rate_of_return'].mean()

-0.037859395103988135

국가별 보유하고 있는 기간이나, 적절한 종목은 다양하게 응용 할 수 있다.
- 적절한 종목 수 , 기간 (3~12)을 다양해게 확인할 수 있겠다.
- 우선주 종목을 제외하면 더 좋겠다.

### Q. 아래의 데이터를 확인하여 종목수, 기간을 변경해보고 수익률을 확인해보자. (이때 만약 6개월기준으로 종목을 선정하고 3개월 보유하고 매도한다면 여기서 6개월 과 3개월은 연속한 데이터를 활용하자.)
- ex) 6개월 기준으로 종목 선정하고 3개월 보유라고 한다면 현재 데이터가 11월 까지의 데이터이므로 1~6월 데이터를 활용하여 종목을 선정하고 7월에 매수하고 11월에 매도

In [ ]:
# 나는 6개월 기준으로 수익률이 높은 기업을 3개월 간 보유하는 알고리즘을 만들겠다.
# 3월~ 9월 기준으로 수익율이 높은 기업을 선정하고 10~12월 간 보유 했을 때 수익률을 계산 (10월 매수, 12월 매도)


data2020
KRX_2021